In [ ]:
# Предметная область: Сейсмология 
# Данные указывают, является ли событие землетрясением или извержением.

In [1]:
import pandas as pd
import sqlite3
import json
import csv

In [14]:
file_path = "earthquake_data.csv"
data_csv = pd.read_csv(file_path, sep=',')
data_csv[0:50]

,mag,place,time,url,magType,longitude,latitude,depth
0,1.30,Central Alaska,1698229886731,https://earthquake.usgs.gov/earthquakes/eventp...,ml,-149.412000,62.998600,66.900000
1,1.05,"10 km NW of The Geysers, CA",1698229719750,https://earthquake.usgs.gov/earthquakes/eventp...,md,-122.836670,38.842499,1.670000
2,0.95,"10 km WNW of Cobb, CA",1698229558030,https://earthquake.usgs.gov/earthquakes/eventp...,md,-122.836166,38.844665,1.840000
3,1.29,"10 km NW of The Geysers, CA",1698229542870,https://earthquake.usgs.gov/earthquakes/eventp...,md,-122.837669,38.843834,1.820000
4,1.48,"10 km WNW of Cobb, CA",1698229465220,https://earthquake.usgs.gov/earthquakes/eventp...,md,-122.837334,38.845501,2.010000
5,1.19,"13 km SE of Bodfish, CA",1698229363910,https://earthquake.usgs.gov/earthquakes/eventp...,ml,-118.395500,35.496167,7.410000
6,1.10,"10 km ESE of Coso Junction, CA",1698228579530,https://earthquake.usgs.gov/earthquakes/eventp...,ml,-117.838833,36.019500,-0.230000
7,4.60,NaN,1698228555411,https://earthquake.usgs.gov/earthquakes/eventp...,mb,-177.705800,-19.243900,413.626000
8,2.03,"10 km E of Pāhala, Hawaii",1698228552880,https://earthquake.usgs.gov/earthquakes/eventp...,md,-155.376160,19.184834,30.799999
9,1.15,"16 km E of Ocotillo, CA",1698228414510,https://earthquake.usgs.gov/earthquakes/eventp...,ml,-115.818167,32.719667,2.980000


In [17]:
def create_csv_table():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    # Создание таблицы для данных из CSV файла
    cursor.execute('''CREATE TABLE IF NOT EXISTS csv_data
                    (mag REAL, 
                     place TEXT, 
                     time INTEGER, 
                     url TEXT, 
                     magType TEXT, 
                     longitude REAL, 
                     latitude REAL, 
                     depth REAL)''')
    # Чтение данных из CSV файла и вставка их в таблицу
    with open('earthquake_data.csv', 'r', encoding='utf-8') as file:
        csv_data = csv.reader(file, delimiter=',')
        next(csv_data)  # Пропуск заголовка
        for row in csv_data:
            if row[1]:  # Проверка наличия данных в колонке "place"
                cursor.execute('INSERT INTO csv_data VALUES(?, ?, ?, ?, ?, ?, ?, ?)', row)
    conn.commit()
    conn.close()


In [18]:
create_csv_table()

In [21]:
# Создание таблицы для данных из JSON файла
def create_json_table():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS json_data
                    (mag REAL, 
                     place TEXT, 
                     time INTEGER, 
                     url TEXT, 
                     magType TEXT, 
                     longitude REAL, 
                     latitude REAL, 
                     depth REAL)''')
    # Чтение данных из JSON файла и вставка их в таблицу
    with open('data.json', 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        for feature in json_data['features']:
            properties = feature['properties']
            coordinates = feature['geometry']['coordinates']
            cursor.execute('INSERT INTO json_data VALUES(?, ?, ?, ?, ?, ?, ?, ?)',
                           (properties['mag'], 
                            properties['place'], 
                            properties['time'], 
                            properties['url'],
                            properties['magType'], 
                            coordinates[0], 
                            coordinates[1], 
                            coordinates[2]))
            
    conn.commit()
    conn.close()             

In [22]:
create_json_table()

In [23]:
# Создание третьей таблицы, объединяющей данные из двух таблиц
def create_json_csv_table():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS merged_data
                    (mag REAL, 
                     place TEXT, 
                     time INTEGER, 
                     url TEXT, 
                     magType TEXT, 
                     longitude REAL, 
                     latitude REAL, 
                     depth REAL)''')
    conn.commit()
    conn.close()  

In [24]:
create_json_csv_table()

In [9]:
# Объединение данных из двух таблиц и вставка их в третью таблицу
def insert_data():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO merged_data SELECT * FROM csv_data UNION SELECT * FROM json_data''')
    conn.commit()
    conn.close()    

In [25]:
insert_data()

In [32]:
# 1. Выборка всех данных из таблицы "merged_data":
def select_all_data():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM merged_data')
    rows = cursor.fetchall()
    conn.close() 
    # Преобразование результатов в формат JSON
    json_data = []
    for row in rows:
        json_data.append({
            'mag': row[0],
            'place': row[1],
            'time': row[2],
            'url': row[3],
            'magType': row[4],
            'longitude': row[5],
            'latitude': row[6],
            'depth': row[7]
        })
    
    # Запись данных в формат JSON
    with open('output_1.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [33]:
select_all_data()

In [35]:
# 2. Выборка данных с простым условием (например, выборка землетрясений с магнитудой больше 5 и сортировка по времени):
def select_data_with_condition():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM merged_data WHERE mag > 5 ORDER BY time')
    rows = cursor.fetchall()
    conn.close()
    # Преобразование результатов в формат JSON
    json_data = []
    for row in rows:
        json_data.append({
            'mag': row[0],
            'place': row[1],
            'time': row[2],
            'url': row[3],
            'magType': row[4],
            'longitude': row[5],
            'latitude': row[6],
            'depth': row[7]
        })
    # Запись данных в формат JSON
    with open('output_2.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [36]:
select_data_with_condition()

In [37]:
# 3. Подсчет количества объектов по условию (например, подсчет землетрясений с магнитудой больше 6):
def count_data_with_condition():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('SELECT COUNT(*) FROM merged_data WHERE mag > 6')
    count = cursor.fetchone()[0]
    conn.close()
    # Преобразование результата в формат JSON
    json_data = {
        'count': count
    }
    # Запись данных в формат JSON
    with open('output_3.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [38]:
count_data_with_condition()

In [39]:
# 4.Группировка данных по магнитуде и подсчет количества землетрясений в каждой группе:
def group_and_count_by_magnitude():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('SELECT mag, COUNT(*) FROM merged_data GROUP BY mag')
    rows = cursor.fetchall()
    conn.close()
    # Преобразование результатов в формат JSON
    json_data = []
    for row in rows:
        json_data.append({
            'magnitude': row[0],
            'count': row[1]
        })
    with open('output_4.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [40]:
group_and_count_by_magnitude()

In [49]:
# 5. Обновление данных в таблице "merged_data" (например, обновление местоположения землетрясения по его url):
def update_location(mag, place, time, new_location):
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('UPDATE merged_data SET place = ? WHERE mag = ? AND place = ? AND time = ?', (new_location, mag, place, time))
    conn.commit()
    conn.close()
    
    # Возвращаем сообщение об успешном обновлении в формате JSON
    json_data = {
        'message': 'Location updated successfully'
    }
    
    # Записываем сообщение в файл JSON
    with open('output_5.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [50]:
update_location(5.8, 'Место', '2023-12-01 10:30:00', 'Новое местоположение')

In [45]:
# 6.Выборка данных с ограничением количества (например, выборка первых 10 землетрясений по времени):
def select_top_10_by_time():
    conn = sqlite3.connect('earthquakes.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM merged_data ORDER BY time LIMIT 10')
    rows = cursor.fetchall()
    conn.close()
    # Преобразование результатов в формат JSON
    json_data = []
    for row in rows:
        json_data.append({
            'mag': row[0],
            'place': row[1],
            'time': row[2],
            'url': row[3],
            'magType': row[4],
            'longitude': row[5],
            'latitude': row[6],
            'depth': row[7]
        })
    with open('output_6.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False)

In [46]:
select_top_10_by_time()